<a href="https://colab.research.google.com/github/cassandraczobit/MentalHealth_LLM/blob/main/LLM_PEFT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import dependencies

In [1]:
# !pip install transformers

In [5]:
pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.6 MB/s eta 0:00:00


In [7]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import math
import transformers
from transformers import AutoModelForSeq2SeqLM

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

ModuleNotFoundError: ignored

### Import Hugging Face model

In [ ]:
# Import mental RoBERTa from Hugging Face
model = AutoModel.from_pretrained("mental-roberta-base")

Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [ ]:
model_checkpoint = 'mental-roberta-base'

# define label maps
id2label = {0: "False", 1: "True"}
label2id = {"False":0, "True":1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

### LoRA (Low Rank Adaptation)


Using LoRA to keep original weights of model frozen and inject the small new trainable parameters with low dimensions matrices.

In [ ]:
# Steps:
# Fine-tuning task (classification), prepare training set (input/output), base model, fine-tune, evaluate
# For classification, need to use transfer learning to replace base model head with classification

In [ ]:
# Import dataset
dataset = load_dataset("NSDUH_2015-2019")
dataset.head()

In [ ]:
# Preprocess data (tokenizer)
tokenizer = AutoTokenizer.from_pretrained("model_checkpoint")

In [ ]:
# Split data for training
dataset = dataset["train"].train_test_split(test_size=0.3)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

In [ ]:
# Define model name and tokenizer
# model_name = "mental-roberta-base"
# tokenizer_name = "bigscience/mt0-large"

In [ ]:
# Create LoRA layer

# task_type = "SEQ_CLS" is for sequence classification
# r is the intrinsic rank of trainable weight matrix
# lora_alpha is similar to learning rate
# lora_dropout is the probability of dropout
# target_modules, apply to query layer only
peft_config = LoraConfig(task_type="SEQ_CLS", r=4, lora_alpha=32, lora_dropout=0.01, target_modules = ['q_lin'])

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

In [ ]:
# Model parameters
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# Hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

# Define training arguments/hyperparameters
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
data_collator =

In [ ]:
# Create parameters to train
train_model = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train model
train_model.train()